In [1]:
# here we load a model and infere test data and check the latent space
import sqlite3
import numpy as np
import comet_ml
# load the database path
from config import settings
from pathlib import Path

settings_proc ='SETTINGS1'
settings_simu = 'SETTINGS1'
root= Path(settings.data.path["processed"])
database_path = (root /settings_simu/settings_simu.lower()).with_suffix('.db')

In [2]:
from psm.utils.data.metadata import get_metadata_processed
metadata= get_metadata_processed(settings_proc, settings_simu)
freq_axis = metadata['freq']

In [3]:
from psm.models.prepare_data import CreateTransformer,PSDDataModule,PSDDataset
# let's create the transformer
transformer = CreateTransformer(database_path, freq=freq_axis, freq_min=0, freq_max=150)
transform_psd = transformer.transform_psd
transform_label = transformer.transform_label
input_dim = transformer.dimension_psd()
dm = PSDDataModule(database_path, transform_psd, transform_label, batch_size=32)
dm.setup()

# Create dataloaders
train_dl = dm.train_dataloader()
val_dl = dm.val_dataloader()
test_dl = dm.test_dataloader()

# Print some details
print(f"Number of training samples: {len(dm.train_dataset)}")
print(f"Number of validation samples: {len(dm.val_dataset)}")
print(f"Number of test samples: {len(dm.test_dataset)}")

# Get a batch of data
for batch in train_dl:
    data, label = batch
    print(f"Shape of data from train_dl: {data.shape}")
    print(f"Shape of label from train_dl: {label.shape}")
    break

(anomaly_level=?) AND stage=? (0, 'train')
(anomaly_level=?) AND stage=? (0, 'test')
Number of training samples: 16000
Number of validation samples: 4000
Number of test samples: 4000
Shape of data from train_dl: torch.Size([32, 385])
Shape of label from train_dl: torch.Size([32])


In [5]:
from psm.models.callbacks_logger import create_callbacks_loggers
from pytorch_lightning import Trainer
from psm.models.vanilla_classification import DenseSignalClassifierModule, DenseSignalClassifier
from torch import nn
callbacks, logger = create_callbacks_loggers()
hyper_params = {'input_dim':input_dim, 'dense_layers':[512,512,248,248,128, 64, 32],
                'dropout_rate':0, 'num_classes':20, 'lr':0.005,
                'batch_norm':True, 'activation':nn.ReLU(), 'l1_reg':1e-4}

model = DenseSignalClassifierModule(**hyper_params)
trainer = Trainer(max_epochs=100, callbacks=callbacks, logger=logger)
trainer.fit(model, dm)
checkpoint_path = trainer.checkpoint_callback.best_model_path
best_model = DenseSignalClassifierModule.load_from_checkpoint(checkpoint_path)
trainer.test(best_model, test_dl)

CometLogger will be initialized in online mode
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/yacinebelhadj/psm-mdof1/db160ddf96f248fab2553a4d0247a1d6



(anomaly_level=?) AND stage=? (0, 'train')


/home/yacine/.local/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/yacine/Documents/PhD/Code/GitProject/PBSHM_mdof/model/model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name      | Type                  | Params
----------------------------------------------------
0 | model     | DenseSignalClassifier | 695 K 
1 | criterion | CrossEntropyLoss      | 0     
2 | train_acc | MulticlassAccuracy    | 0     
3 | val_acc   | MulticlassAccuracy    | 0     
4 | test_acc  | MulticlassAccuracy    | 0     
----------------------------------------------------
695 K     Trainable params
0         Non-trainable params
695 K     Total params
2.782     Total estimated model params size (MB)


(anomaly_level=?) AND stage=? (0, 'test')


Sanity Checking: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

torch.Size([4000]) torch.Size([4000])


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │            1.0            │
│         test_loss         │     2.112715721130371     │
└───────────────────────────┴───────────────────────────┘

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/yacinebelhadj/psm-mdof1/db160ddf96f248fab2553a4d0247a1d6
COMET INFO:   Metrics:
COMET INFO:     test_acc  : 1.0
COMET INFO:     test_loss : 2.112715721130371
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO:   Parameters:
COMET INFO:     batch_norm   : True
COMET INFO:     bias         : True
COMET INFO:     dense_layers : [512, 512, 248, 248, 128, 64, 32]
COMET INFO:     dropout_rate : 0
COMET INFO:     input_dim    : 385
COMET INFO:     l1_reg       : 0.0001
COMET INFO:     lr           : 0.005
COMET INFO:     num_classes  : 20
COMET INFO:     temperature  : 1.0
COMET INFO:   Uploads:
C

[{'test_loss': 2.112715721130371, 'test_acc': 1.0}]